# SCRIPT TO ANALIZE TWEETS FROM AACREA FOLLOWERS #

### create dfTweets from csv

In [9]:
#analyze files
#creates a db of tweets
from os import listdir
import pandas as pd
fileList = listdir("tweets")
fileList = [f for f in fileList if f[0]!="."]
print(str(len(fileList)) + ' users analyzed.')

totalTweets = 0
fileListWContent = []
dfTweets = []
for fileName in fileList:
    df = pd.read_csv("tweets/" + fileName, sep=";;")
    if df.shape[0]>1:
        fileListWContent.append(fileName)
        totalTweets = totalTweets + len(df["text"])
        # create a df with date, user, text, sentiment
        dfTweetsTemp = df[['date','text']]
        dfTweetsTemp['user'] = fileName[:(len(fileName)-4)]
        dfTweets.append(dfTweetsTemp)

dfTweets = pd.concat(dfTweets, axis=0)

#convert to date
dfTweets["date"] = pd.to_datetime(dfTweets["date"])

#remove records outside the limits
dfTweets = dfTweets.loc[dfTweets['date'] < '2016-09-01']
dfTweets = dfTweets.loc[dfTweets['date'] >= '2014-05-01']

dfTweets['month'] = dfTweets.date.map(lambda x: x.strftime('%Y-%m'))

#table with occurrencies
monthlyTable = dfTweets["date"].groupby(dfTweets.date.dt.to_period("M")).agg('count')
months = list(monthlyTable.index)

#list of months
months = [str(month) for month in months]
dfTweets = dfTweets.reset_index()

3066 users analyzed.


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### auxiliary functions

In [8]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def tokenize(text):
    tokens = word_tokenize(text.lower())
    
    return tokens

def intersection(wordList1, wordList2):
    temp = list(set(wordList1).intersection(set(wordList2)) )
    out = 0
    if len(temp)>0:
        out=1
    return out

def intersection2(wordList1, wordList2):
    temp = list(set(wordList1).intersection(set(wordList2)) )
    out = 0
    if len(temp)>1:
        out=1
    return out


### identify relevant tweets by relevante2.txt 

In [9]:
#relevante
relCorpus = list(pd.read_csv('relevante2.txt', header=None)[0])
relCorpus2 = list(pd.read_csv('relevante3.txt', header=None)[0])

relVector = []
relVector2 = []
for t in dfTweets['text']:
    relVector.append(intersection(tokenize(str(t)), relCorpus))
    relVector2.append(intersection2(tokenize(str(t)), relCorpus2))
    
dfTweets['relevant'] = relVector
dfTweets['relevant2'] = relVector2

### sentiment by Google (do not run)

In [25]:
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

client = language.LanguageServiceClient()

sentimentDict = {}
for i, row in dfTweets.iterrows():
    if dfTweets.iloc[i, dfTweets.columns.get_loc('relevant')]==0 and dfTweets.iloc[i, dfTweets.columns.get_loc('relevant2')]==1:
        print(i)
        #sentimentDict[i]= dfTweets.iloc[i, dfTweets.columns.get_loc('text')]
        textToAnalize = dfTweets.iloc[i, dfTweets.columns.get_loc('text')]
        document = types.Document(
            content=textToAnalize,
            type=enums.Document.Type.PLAIN_TEXT,
            language='es')

        annotations = client.analyze_sentiment(document=document, encoding_type='UTF8')
        
        sentimentDict[i] = annotations
        
import pickle
with open('sentimentDict3.pickle', 'wb') as handle:
    pickle.dump(sentimentDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

1
4
59
79
126
130
156
501
720
1410
1441
1682
1715
1731
1733
3130
3161
3462
3497
3533
3538
3541
3551
3557
3573
3578
3599
3638
3666
4552
4674
4730
4775
4801
4831
5736
6031
6032
6262
6291
6329
6440
6762
6805
7058
7754
7818
7823
7828
7991
7992
7995
7999
8000
8003
8004
8014
8015
8021
8032
8033
8039
8042
8192
8248
8256
8257
8259
8336
8342
8796
9031
9606
9737
10075
10353
10567
10578
10589
10611
10624
10628
10631
10648
10677
10715
10786
10796
10819
10825
10827
10867
10935
10971
11101
11106
11152
11166
11168
11226
11249
11250
11251
11423
11980
11997
12000
12031
12258
12284
12366
12373
12392
12405
12415
12416
12418
12430
12432
12435
12436
12892
12965
13117
14146
14163
14191
14198
14208
14209
14221
14786
14787
14830
14896
14927
14940
14941
14956
14965
14976
14998
15008
15017
15019
15058
15085
15090
15095
15106
15108
15120
15147
15148
15198
15609
15611
15666
16257
17425
17444
17459
17469
17518
17545
17592
17692
17948
18262
18491
18627
19302
19508
19511
19513
19652
19655
19871
20127
20130
20138
201

147845
147846
147847
147851
147852
147853
147854
147857
147859
147860
147861
147862
147863
147865
147866
147868
147870
147872
147873
147874
147876
147880
147882
147884
147888
147891
147892
147893
147894
147896
147897
147898
147899
147900
147902
147907
147911
147912
147914
147921
147923
147924
147927
147930
147938
147943
147944
147972
147988
147991
147992
148002
148003
148127
148135
148156
148158
148174
148199
148462
148465
148540
149140
149219
149224
149881
149883
149887
149938
151241
151497
151804
151813
151815
151818
151821
151843
151846
151847
151851
151862
151867
151871
151881
151888
151891
151894
151921
151936
151937
151954
151977
151978
151983
151990
151999
152002
152016
152017
152020
152022
152023
152041
152059
152060
152064
152065
152066
152067
152075
152086
152088
152091
152102
152108
152111
152112
152113
152119
152120
152121
152129
152222
152889
153817
153967
153979
154115
154183
155320
155573
156285
156293
156307
156370
156741
157636
157653
157714
157716
158158
158402
158404

### sentiment by Watson (do not run)

In [26]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, SentimentOptions

natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2019-07-12',
    iam_apikey='O3dkk_tabO5M96yk6vSslzFKvj47CLYm6XywtZiG7Jdl',
    url='https://gateway.watsonplatform.net/natural-language-understanding/api'
)

sentimentDict2 = {}
for i, row in dfTweets.iterrows():
    if dfTweets.iloc[i, dfTweets.columns.get_loc('relevant')]==0 and dfTweets.iloc[i, dfTweets.columns.get_loc('relevant2')]==1:
        #print(i)
        try:
            response = natural_language_understanding.analyze(
            text=dfTweets.iloc[i, dfTweets.columns.get_loc('text')],
            #text='Hoy, con el gobernador de la provincia, Maurice Closs, se inaugura oficialmente la Feria Forestal Argentina. Los esperamos a las 18:00 hs.'
            language="es",
            features=Features(sentiment=SentimentOptions())).get_result()
            sentimentDict2[i] = response
            #print(dfTweets.iloc[i, dfTweets.columns.get_loc('text')])
            #print(response)
        except:
            print("error")

import pickle
with open('sentimentDict4.pickle', 'wb') as handle:
    pickle.dump(sentimentDict2, handle, protocol=pickle.HIGHEST_PROTOCOL)

### sentiment by NLTK (do not run)

In [ ]:
#todo

### load pickle

In [29]:
import pickle
import pandas as pd
with open('sentimentDict.pickle', 'rb') as handle:
    sentimentDict = pickle.load(handle)
    
with open('sentimentDict2.pickle', 'rb') as handle:
    sentimentDict2 = pickle.load(handle)

### mark sentiment in dfTweets

In [30]:
import numpy as np
#dfTweets['score'] = np.nan
#dfTweets['magnitude'] = np.nan
#dfTweets['scoreWatson'] = np.nan

for j in sentimentDict:
    #print(j, sentimentDict[j].document_sentiment.score,sentimentDict[j].document_sentiment.magnitude)
    dfTweets.iloc[j, dfTweets.columns.get_loc('score')] = sentimentDict[j].document_sentiment.score
    dfTweets.iloc[j, dfTweets.columns.get_loc('magnitude')] = sentimentDict[j].document_sentiment.magnitude
    try:
        dfTweets.iloc[j, dfTweets.columns.get_loc('scoreWatson')] = sentimentDict2[j]['sentiment']['document']['score']
    except:
        pass

### save dfTweets

In [31]:
dfTweets.to_csv('dfTweets5.csv')

### save dfTweets

In [5]:
dfTweets = pd.read_csv('dfTweets5.csv')

### plot results

In [2]:
#load encuesta
import pandas as pd
encuesta = pd.read_excel('encuesta resultados.xlsx')

In [6]:
for i, row in dfTweets.iterrows():
    if row['relevant']==1:
        print(row['text'],row['score'])

Clave Wi Fi: $$presidencia Mortal!! Jajajaja https:// twitter.com/eameook/status /753258551179698177 … 0.6000000238418579
El presidente de la Feria Forestal Argentina, Josué Barrios Ledesma, en el lanzamiento oficial. https://www. youtube.com/watch?v=jhRPO6 QbnSQ&feature=youtu.be … 0.10000000149011612
Hoy se lanzó la undécima Feria Forestal Argentina. El acto se realizó en la sala de Situación de Casa de Gobierno. pic.twitter.com/WBNcWBzth6 0.0
Con la presencia del gobernador de Misiones, Maurice Closs, quedó inaugurada oficialmente la 10°de la Feria Forestal pic.twitter.com/0xai70TdUj 0.30000001192092896
@noticiasdel6 Hoy, con el gobernador de Misiones, Maurice Closs, se inaugura oficialmente la Feria Forestal Argentina, a las 18:00 horas. 0.4000000059604645
@milciades1983 Hoy, con el gobernador de Misiones, Maurice Closs, se inaugura oficialmente la Feria Forestal Argentina, a las 18:00 horas. 0.4000000059604645
@lauvivanco Hoy, con el gobernador de Misiones, Maurice Closs, se inaugu

Los caminos de la producción son una estafa igual que el gobierno de Uribarri -0.8999999761581421
Incompetencia del impresentable del ministro de la producción de la provincia de Entre Rios -0.8999999761581421
#DEPORTES : Ameal dará una charla sobre su experiencia como presidente de Boca Juniors. INFO: http://www. frcon.utn.edu.ar/wpfrcon/?p=8091 pic.twitter.com/yH0A38MyMa 0.10000000149011612
El Gobierno Y La Uier Firmaran Este Martes El Acta Constitutiva Del Observatorio De ... http://www. camposrenovables.com.ar/2014/05/el-gob ierno-y-la-uier-firmaran-este-martes-el-acta-constitutiva-del-observatorio-de-energias-renovables/ … #Campos #Renovables #Blog 0.30000001192092896
Restricciones Al Gas: El Gobierno Endurece Controles Sobre Industrias Y Petroleras http://www. camposrenovables.com.ar/2014/05/restri cciones-al-gas-el-gobierno-endurece-controles-sobre-industrias-y-petroleras/ … #Campos #Renovables #Blog 0.10000000149011612
Situación Actual Y Perspectivas Del Futuro Del Sector De La

@Maxigraffi esa es tu segunda casa. Clases de Economía 1 doy de 9 a 13hs todos los lunes. 0.4000000059604645
Para Cavallo, el Gobierno puede transformar al fallo de la Corte en una oportunidad http:// m.cronista.com/economiapoliti ca/Para-Cavallo-el-Gobierno-puede-transformar-al-fallo-de-la-Corte-en-una-oportunidad-20160819-0099.html … vía @cronistacom 0.20000000298023224
@LaMiradaRG @canal26noticias Alfredo Leuco...." Cristina fue lo peor de su Gobierno ...." -0.4000000059604645
Que Ministro...Sencillo, Honesto, Inteligente...y Sobre Todo Prestigio Intachable....Siga Adelante que Lo Bancamos.. https:// twitter.com/todonoticias/s tatus/757748150375284736 … 0.5
@telenoche EN INSTANTES LA TRUCHA DE PLATINA PARA EL GOBERNADOR DE MISIONES MAURICE CLOSS 0.30000001192092896
Hacia aca nos quiera llevar Nuestro Presidente @mauriciomacri ...Donde estan los Paises del Primer Mundo... https:// twitter.com/wef/status/739 103746999881728 … 0.10000000149011612
El Gobierno lanza plan para incorporar 

En la cubre del clima impulsan una transición ordenada hacia una economía baja en carbono http://www. elsemiarido.com/en-la-cubre-de l-clima-impulsan-una-transicion-ordenada-hacia-una-economia-baja-en-carbono/ … 0.4000000059604645
[MINISTERIO DEL CAMPO] Avanza el Plan Pasturas para Mi Provincia en el norte puntano http:// agenciasanluis.com/notas/2015/10/ 28/avanza-el-plan-pasturas-para-mi-provincia-en-el-norte-puntano/ … vía @agenciasanluis 0.0
El Ministerio del Campo sembrará buffel y llorón para 99 productores http:// agenciasanluis.com/notas/2015/10/ 05/poggi-en-los-ultimos-30-anos-la-obra-publica-transformo-los-campos-de-san-luis/ … @revistachacra @INTAsanluis @lahoradelcampo 0.20000000298023224
El Ministerio del Campo sembrará buffel y llorón para 99 productores http:// agenciasanluis.com/notas/2015/10/ 05/poggi-en-los-ultimos-30-anos-la-obra-publica-transformo-los-campos-de-san-luis/ … @Palabra_Rural @infocampoweb @pmotivar 0.10000000149011612
El Ministerio del Campo sembrará bu

Buena política para imitar...un estado pensando y actuando en las potencialidades de su economía regional. https:// twitter.com/pnutsusa/statu s/663737907560226816 … 0.30000001192092896
Felicitaciones!!! CIA Cabrera e @intaargentina #Orgullo del sector economía regional apostando siempre a crecer!!! https:// twitter.com/CamaradelMani/ status/644463809063600128 … 0.4000000059604645
Gobierno argentino ratifica cooperación para el Desarrollo Sostenible con #PNUDArgentina http:// bit.ly/2a5D1Am https:// twitter.com/PNUDArgentina/ status/756477056112857088 … 0.699999988079071
2° día de la Jornada Nac.de #ForrajesConservados abordamos la situación de la lechería y estrategias p/ afrontarla pic.twitter.com/TL0rW1rsGA 0.5
En la Jornada de #ForrajesConservados del @intaargentina se abordará la situación desesperante de los granos afectados x humedad. -0.6000000238418579
El @intaargentina tiene nuevas autoridades: Amadeo Nicora (Presidente) y Mariano Bosch (Vice): http:// intainforma.inta.gov.ar

...

@carve850 insólita no se la robaron y al señor presidente se le desfilaron los cuchillos para cortar las manos que prometio -0.4000000059604645
@dcastro65 @AIntransigente el presidente pobre usando camperas the North face igual a la de lázaro baez,será una coima -0.10000000149011612
@pedro_silva54 @ObservadorUY Pero el presidente pobre fue al brasil para salvar a dilma 0.0
Alguno se esta informando de que en cada país existe la constitución(dentro de la constitución todo,dijo el presidente fuera nada) -0.5
@burrosuy @AlfredoGarcia09 @ObservadorUY El presidente pobre que donó 200.000 U$S para plazita en España, no le quedó nada para los de aqui -0.8000000119209291
Completamente de acuerdo no podemos tener a un ministro qué tiene sangre en sus manos hay que contarle eso a la juventud -0.30000001192092896
Jeje quien estuvo a cargo de la economía 0.6000000238418579
El problema es que al presidente se le perdió el arma con la cual iva a cortar las manos 0.0
El gobierno no necesita acuerdos 

Ya lo dijimos durante diciembre y enero Consorcio regional Marcos Juárez: “La situación es gravísima” http://www. agrovoz.com.ar/clima/consorci o-regional-marcos-juarez-la-situacion-es-gravisima … vía @agrovoz 0.0
Hacía rato q no compartía pero la situación lo amerita. #lluvias #BuenosAires #Inundaciones #Argentina pic.twitter.com/HYQ5AtKN2M 0.5
Se conoció el fundamento bajista de los fondos: el gobierno de EE.UU. comenzaría a reducir el uso de etanol maicero en la matriz energética 0.0
Trigo: Perspectivas del presidente de @AAPROTrigo http://www. fyo.com/noticia/149587 /trigo-mejor-negocio-cultivos#.VWRfHaOphYk.twitter … 0.10000000149011612
@AteneosCRArg @diariodecuyoweb @fmercuriali @AldoBonaveri La agrotecnología puede majorar la eficiencia si la política sectorial lo permite 0.0
Ya en General Pico la situación es otra con rendimientos mas que interesantes. #Soja #LaPampa pic.twitter.com/HN2mONdh34 0.30000001192092896
Vuelo Realizado por @estimacionesbc para el informe de Situación 

Entre Rios: Ministerio de Trabajo se pronunció a favor de APAMA sobre situación laboral de cosecheros La máxima... http:// fb.me/1omj6FIRD 0.5
La región mesopotámica comienza temporada de arándanos El presidente de APAMA confirmó que se empleará la misma... http:// fb.me/1lS2ClJ1J 0.0
Arándanos: Reporte de pretemporada 2014 El reporte de Pretemporada es un cuadro de situación por Región que... http:// fb.me/751JIVOIh -0.20000000298023224
#Arándanos : #Tucumán arrancaría su campaña en Septiembre El gobierno de esa provincia espera para este año, que... http:// fb.me/33YPuQ6Ue -0.699999988079071
#APAMA reclama al Gobierno créditos anti-granizo La Asociación de Productores de Arándanos de la Mesopotamia... http:// fb.me/2ecL75sMD 0.10000000149011612
Estado de la producción del arándano argentino: total estabilidad El Ministerio de Agricultura de la Nación con... http:// fb.me/1liLupvSU 0.10000000149011612
#Arándanos #Argentinos conquistan el #MercadoAsiático : Carlos Stabile, presidente d

@fabitaaraujo Piden BIS de gobierno. -0.8999999761581421
http://www. ayt.com.ar/una-mirada-sob re-la-proxima-campana-agricola/ … El cambio de gobierno contribuye para recuperar las ganas de #invertir Pero hay que analizar todas las variables 0.30000001192092896
La vitivinicultura es la economía regional que va a salir más rápidamente cuando se ordene la macro del país. http://www. lanacion.com.ar/1843225-negoci os-que-vuelven-la-industria-del-vino-lidera-un-incipiente-proceso-de-fusiones-y-adquisiciones-de-empresas … 0.30000001192092896
http://www. lanacion.com.ar/1817291-hace-f alta-un-plan-para-que-el-campo-y-las-agroindustrias-se-desarrollen … Palabras del Presidente de la @SociedadRural 0.20000000298023224
"El gobierno necesita dólares para el año 2016. El campo necesita una devaluación para mejorar su rentabilidad" http:// agroeducacion.com/web/noticias/1 098 … -0.20000000298023224
A&T en #BASF2015CÓRDOBA . @GabrielTelleri2 habló sobre "Anál. de la situación productiva. Estrat. pa

KeyboardInterrupt: 

In [22]:
i = 1
dfTweets.iloc[i, dfTweets.columns.get_loc('relevant')]==0 and dfTweets.iloc[i, dfTweets.columns.get_loc('relevant2')]==1

True

In [8]:
import numpy as np
df2plot = pd.pivot_table(dfTweets, values='score', index=['user'],columns=['month'], aggfunc=np.mean).mean()
df2plotCount = pd.pivot_table(dfTweets, values='relevant', index=['user'],columns=['month'], aggfunc=np.sum).sum()
df2plot2 = pd.pivot_table(dfTweets, values='scoreWatson', index=['user'],columns=['month'], aggfunc=np.mean).mean()

import numpy as np
import matplotlib.pyplot as plt

# Create some mock data
t = range(len(months))
data1 = df2plot.values
#data2 = df2plotCount.values
data2 = encuesta['Economía']

data3 = df2plot2.values

fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('date')
ax1.set_ylabel('sentiment', color=color)
ax1.plot(t, data1, color=color)
ax1.plot(t, data3, color='blue')

ax1.tick_params(axis='y', labelcolor=color)
#ax1.tick_params(axis="x", range(len(months)), months, rotation='vertical')

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('quantity', color=color)  # we already handled the x-label with ax1
df2plotCount.plot(kind="bar", alpha=0.5, color = 'blue')
#ax2.plot(t, data2, color=color)
ax1.tick_params(axis='x',  rotation=90)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

NameError: name 'months' is not defined

In [ ]:
for i in range(len(data1)):
    print (months[i],data1[i],data3[i])

In [ ]:
dfTweets[~pd.isnull(dfTweets['score'])]

In [ ]:
for i in df2plot2.values:
    print(i)

In [ ]:
sentimentDict2[82542]['sentiment']['document']['score']

In [ ]:
from ibm_watson.natural_language_understanding_v1 \
    import Features, EntitiesOptions, KeywordsOptions
    
response = natural_language_understanding.analyze(
    text='IBM is an American multinational technology company '
    'headquartered in Armonk, New York, United States, '
    'with operations in over 170 countries.',
    features=Features(
        entities=EntitiesOptions(sentiment=True, limit=2))).get_result()

print(json.dumps(response, indent=2))

In [ ]:
df2plot = pd.pivot_table(dfTweets, values='score', index=['user'],columns=['month'], aggfunc=np.mean).mean()
#df2plotCount = dfTweets.groupby('month').agg('count')
import matplotlib.pyplot as plt
plt.plot(df2plot.values)
plt.ylabel('sentiment')
plt.xticks(range(len(months)), months, rotation='vertical')
plt.show()

In [ ]:
pd.pivot_table(dfTweets, values='score', index=['user'],columns=['month'], aggfunc=np.count).mean()

In [12]:
i

176578

In [6]:
dfTweets

,index,date,text,user,month,relevant,score,magnitude,scoreWatson
0,0,2016-08-22 17:27:00,Publiqué una nueva foto en Facebook http:// fb...,Siclorural,2016-08,0,NaN,NaN,NaN
1,1,2016-08-10 13:01:00,Gracias Campo y Ecología por la nota realizada...,Siclorural,2016-08,0,NaN,NaN,NaN
2,2,2016-08-10 11:29:00,...Año 2016! Expo Recicla! http:// fb.me/5L9gE...,Siclorural,2016-08,0,NaN,NaN,NaN
3,3,2016-08-08 17:08:00,"#SiCuidasTuCampo , hacelo con @Siclorural ! Co...",Siclorural,2016-08,0,NaN,NaN,NaN
4,4,2016-08-05 17:10:00,Mañana por la mañana Siclo Rural en Radio Mitr...,Siclorural,2016-08,0,NaN,NaN,NaN
5,5,2016-08-05 17:09:00,Año 2015 !! http:// fb.me/PofWRpi5,Siclorural,2016-08,0,NaN,NaN,NaN
6,6,2016-08-05 14:27:00,Que tengan un buen fin de semana #SicloRural,Siclorural,2016-08,0,NaN,NaN,NaN
7,7,2016-08-04 11:31:00,Año 2014! Participación en el Congreso Interna...,Siclorural,2016-08,0,NaN,NaN,NaN
8,8,2016-08-03 16:08:00,"Estamos de festejo , Cumplimos 5 años y vamos ...",Siclorural,2016-08,0,NaN,NaN,NaN
9,9,2016-08-03 11:33:00,La Hora del Mate - Año 2013! http:// fb.me/2ov...,Siclorural,2016-08,0,NaN,NaN,NaN


In [ ]:
dfTweets["score"].groupby(['user','M']).size().reset_index().groupby('col2')[[0]].max()

sentTable = dfTweets["score"].groupby(dfTweets.date.dt.to_period("M")).agg('mean')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(sentTable.values)
plt.ylabel('sentiment')
plt.show()

In [ ]:
dfTweets["score"].groupby(['user',dfTweets.date.dt.to_period("M")])

# sentiment by google

In [ ]:

sentimentDict = {}
for i, row in dfTweets.iterrows():
    if dfTweets.iloc[i, dfTweets.columns.get_loc('relevant')]==1:
        print(i)
        #sentimentDict[i]= dfTweets.iloc[i, dfTweets.columns.get_loc('text')]
        textToAnalize = dfTweets.iloc[i, dfTweets.columns.get_loc('text')]
        document = types.Document(
            content=textToAnalize,
            type=enums.Document.Type.PLAIN_TEXT,
            language='es')

        annotations = client.analyze_sentiment(document=document, encoding_type='UTF8')
        
        sentimentDict[i] = annotations

In [ ]:
#calculate sentiment by sdal bag of words
import numpy as np
#create a dictionary
sdal = pd.read_csv('sdal.csv')
sdalDict = {}
for idx, row in sdal.iterrows():
    sdalDict[row['palabra']] = row['media_agrado']


dfTweets['sentimentSdal'] = np.nan

dfTweets = dfTweets.reset_index(drop=True)

for i, row in dfTweets.iterrows():
    sentByWord = []
    tokenizedText = tokenize(str(row['text']))
    for word in tokenizedText:
        try:
            sentByWord.append(sdalDict[word])
        except:
            pass
    #print(i,np.mean(sentByWord))
    dfTweets.iloc[i, dfTweets.columns.get_loc('sentimentSdal')] = np.mean(sentByWord)   

# calculate relevance by bag of words 

In [ ]:
#relevante
relCorpus = list(pd.read_csv('relevante2.txt', header=None)[0])

relVector = []
for t in dfTweets['text']:
    relVector.append(intersection(tokenize(str(t)), relCorpus))

dfTweets['relevant'] = relVector
#dfTweets.iloc[1, dfTweets.columns.get_loc('text')]= 'aaa'

In [ ]:
dfTweets.to_csv('dfTweets.csv')

In [ ]:
sentimentDict = {}

for i, row in dfTweets.iterrows():
    if dfTweets.iloc[i, dfTweets.columns.get_loc('relevant')]==1:
        print(i)
        #sentimentDict[i]= dfTweets.iloc[i, dfTweets.columns.get_loc('text')]
        textToAnalize = dfTweets.iloc[i, dfTweets.columns.get_loc('text')]
        document = types.Document(
            content=textToAnalize,
            type=enums.Document.Type.PLAIN_TEXT,
            language='es')

        annotations = client.analyze_sentiment(document=document, encoding_type='UTF8')
        
        sentimentDict[i] = annotations

In [ ]:
dfTweets['score'] = np.nan
dfTweets['magnitude'] = np.nan

for j in sentimentDict:
    #print(j, sentimentDict[j].document_sentiment.score,sentimentDict[j].document_sentiment.magnitude)
    dfTweets.iloc[j, dfTweets.columns.get_loc('score')] = sentimentDict[j].document_sentiment.score
    dfTweets.iloc[j, dfTweets.columns.get_loc('magnitude')] = sentimentDict[j].document_sentiment.magnitude

In [ ]:
import pickle
with open('sentimentDict.pickle', 'wb') as handle:
    pickle.dump(sentimentDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('sentimentDict.pickle', 'rb') as handle:
    b = pickle.load(handle)

print (sentimentDict == b)


In [ ]:
import pickle
with open('sentimentDict.pickle', 'rb') as handle:
    sentimentDict = pickle.load(handle)

# calculate sentiment by google


In [ ]:
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

In [ ]:
document = types.Document(
            content='Se cae más el dólar, contra una inflación más elevada. Salvador Di Stefano https://www. facebook.com/agropecuariasa njusto/posts/1029444323802257',
            type=enums.Document.Type.PLAIN_TEXT,
            language='es')

annotations = client.analyze_sentiment(document=document, encoding_type='UTF8')

In [ ]:
sentimentDict ={}
sentimentDict [3] = annotations

In [ ]:
sentimentDict[3]

In [ ]:

tokenizedText = tokenize(tt) 
for word in tokenizedText:
        try:
            sentByWord.append(sdalDict[word])
            print(word,sdalDict[word])
        except:
            pass

In [ ]:
sentimentDict

In [ ]:
sum(dfTweets['relevant'])

In [ ]:
dfTweets['text'][4]

In [ ]:
sentTable = dfTweets["sentiment"].groupby(dfTweets.date.dt.to_period("M")).agg('mean')

In [ ]:
monthlyTable

In [ ]:
sentTable

In [ ]:
pivotTable = pd.pivot_table(dfTweets[dfTweets.relevant==1], values='score', index='user',columns='month', aggfunc=np.mean)

In [ ]:
pivotTable.to_csv('pivotTable.csv')

In [ ]:
for t in dfTweets[dfTweets.relevant==1]['text']:
    print(t)

In [ ]:
list(dfTweets['date'])

In [ ]:
list(dfTweets['date']).date_time.map(lambda x: x.strftime('%Y-%m'))
#dfTweets['month'] = df.apply (lambda row: label_race(row), axis=1)

In [ ]:
sentimentDict

In [ ]:
dfTweets = pd.concat(dfTweets, axis=1)

In [ ]:
fileName[:(len(fileName)-4)]

In [ ]:
# create a df with date, user, text, sentiment
dfTweetsTemp = df[['date','text']]
dfTweetsTemp['user'] = fileName[:(len(fileName)-4)]

In [ ]:
dfTweets

In [ ]:
df

In [ ]:
import pandas as pd
print(len(fileList))
totalTweets = 0
for fileName in fileList:
    if (fileName[0]!="."):
        print(fileName)
        df = pd.read_csv("tweets/" + fileName, sep=";;")

        totalTweets = totalTweets + len(df["text"])
        #for tweet in df["text"]:
        
        #print(tweet)
        #print("-----------------------------------------------")

totalTweets/len(fileList)*7500

In [ ]:
import spacy
spacy.load('es_core_news_sm')
from spacy.lang.es import Spanish
parser = Spanish()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        #elif token.orth_.startswith('@'):
        #   lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in es_stop]
    #tokens = [get_lemma(token) for token in tokens]
    return tokens

In [ ]:
totalTweets

In [ ]:
import nltk
nltk.download('stopwords')
es_stop = set(nltk.corpus.stopwords.words('spanish'))


In [ ]:
text_data = []
for fileName in fileList:
    if (fileName[0]!="."):
        #print(fileName)
        df = pd.read_csv("tweets/" + fileName, sep=";;")

        for tx in df["text"]:
            
            try:
                tokens = prepare_text_for_lda(tx)
                text_data.append(tokens)
            except:
                pass

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

In [ ]:
import gensim
NUM_TOPICS = 15
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

In [ ]:
corpus[0]

In [ ]:
months

In [ ]:
import collections
wordcount = {}
# To eliminate duplicates, remember to split by punctuation, and use case demiliters.
for sen in text_data:
    for word in sen:
        if word not in wordcount:
            wordcount[word] = 1
        else:
            wordcount[word] += 1
            

In [ ]:
word_counter = collections.Counter(wordcount)
for word, count in word_counter.most_common(30):
    print(word, ": ", count)